In [4]:
# -*- coding: utf-8 -*-
import os
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
import pickle
import seaborn as sns
import math
import re


from matplotlib import pyplot as plt 

In [1]:
data_folder = "/home/vietphan/Downloads/fbcrawl/Data-Celeb-Nov/"
source = data_folder+"model/"+"source.txt"
filename = data_folder+"model/"+"corpus.txt"
rating_file = data_folder+"model/"+"rating.txt"


In [5]:
source_name = []
corpus = []
page_id = []
doc_count = 0
rating = []
with open(filename, 'r') as f:
    line = f.readline()
    while(line):
        if(line[0]!="*"):
            corpus.append(line)
            # doc_count = doc_count +1
        line = f.readline()
with open(rating_file, 'r') as f2:
    line = f2.readline()
    while(line):
        if(line[0]=="*"):
            page_id.append(doc_count)#-len(page_id))
        else:
            rating.append(np.int64(line))
            doc_count = doc_count +1
        line = f2.readline()
# print(page_id)

with open(source, 'r') as f2:
    line = f2.readline()
    while(line):
        if(line[0]=="*"):
#             page_id.append(doc_count-len(page_id))
            a = 6
        else:
            source_name.append(line)
            doc_count = doc_count +1
        line = f2.readline()

In [6]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names()
# print(vocab)
# for i in range(len(page_id)-1):
#     sum = 0
#     t_corpus = []
#     for j in range(page_id[i],page_id[i+1]):
            



# for j in range(page_id[-1],len(rating)):
#     sum = sum + rating[j]


In [7]:
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils


In [8]:
def remove_stop_words(line,stop_words,stop_words_no_accent):
    words = line.split()
    mid = 0
    line = ""
    for count,word in enumerate(words):
        word_no_accent = ViUtils.remove_accents(u""+word)
        left = 0
        right = len(stop_words)-1
        while(int((right-left)/2)>0):
            mid = int((right+left)/2)
            # print(word_no_accent,stop_words_no_accent[mid],(word_no_accent>stop_words_no_accent[mid]),left,right)
            if(word_no_accent>stop_words_no_accent[mid]):
                left = mid
            elif(word_no_accent<stop_words_no_accent[mid]):
                right = mid
            elif((word_no_accent==stop_words_no_accent[mid])):
                break
        if(word_no_accent != stop_words_no_accent[mid]):
            # print(word,stop_words[mid])
            line = line+word+" "
        else:
            if(mid>=1 and mid<=(len(stop_words)-1)):
                check = 0
                for i in range(mid-1,mid+1):
                    if(word==stop_words[i]):
                        check = 1
                        break
                if(check == 0):
                    line = line+word+" "
            elif(mid <=1):
                check = 0
                for i in range(0,mid+1):
                    if(word==stop_words[i]):
                        check = 1
                        break
                if(check == 0):
                    line = line+word+" "
            elif(mid >= (len(stop_words)-4)):
                check = 0
                for i in range(mid - 5,mid):
                    if(word==stop_words[i]):
                        check = 1
                        break
                if(check == 0):
                    line = line+word+" "
        # else:
        #     print(word)
    return line
    
    

In [9]:
def remove_trash(doc, low_IDF, low_IDF_no_acc):
    line = ''
    words = doc.split()
#     print(low_IDF)
    if(len(low_IDF)!=0):
        for low in low_IDF:
            for word in words:
                if(word == low):
                    words.remove(word)
    for word in words:
        line = line + word + " "
    return line
        

In [10]:
print(len(page_id))

0


In [11]:
n = 80
i=n

for i in range(len(page_id)):
    print("Processing "+str(i))
#     print(page_id[i+1])
    k=i
    t_corpus = []
    low_IDF = []
    low_IDF_no_acc = []
    if(i<len(page_id)-1):
        for j in range(page_id[i],page_id[i+1]):
            t_corpus.append(corpus[j])
    else:
        for j in range(page_id[-1],len(rating)):
            t_corpus.append(corpus[j])
    if(len(t_corpus)>0):
        vectorizer2 = CountVectorizer()
        Y = vectorizer2.fit_transform(t_corpus)
        vocab_page = vectorizer2.get_feature_names()
        vector_page = Y.toarray()

        for i, word in enumerate(vocab_page):
                word = re.sub(r'\#', '', word)
                word = re.sub(r'\>', '', word)
                count_word_in_corpus = 0
                ind = vocab_page.index(word) 
                for j in range(len(vector_page)):
                    if(vector_page[j][ind]!=0):
                        count_word_in_corpus+=1
                IDF = math.log(len(vector_page)/count_word_in_corpus)
        #         print(word+" "+str(IDF))
                if(IDF<1):
                    low_IDF.append(word)
                    low_IDF_no_acc.append(ViUtils.remove_accents(u""+word))

        # test_corpus = []
        if(k<len(page_id)-1):
            for j in range(page_id[k],page_id[k+1]):
                corpus[j] = remove_trash(corpus[j],low_IDF,low_IDF_no_acc)
        else:
            for j in range(page_id[-1],len(rating)):
                corpus[j] = remove_trash(corpus[j],low_IDF,low_IDF_no_acc)






#     test_corpus.append(line)

In [12]:
done_processing = data_folder+"model/"+"done_processing.txt"
with open(done_processing, 'w') as f:
    for line in corpus:
        if(len(line)!=0):
            f.write(line)
            f.write("\n")
